<a href="https://colab.research.google.com/github/unt-iialab/INFO5731_Spring2020/blob/master/Assignments/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [Apple iPhone 11](https://www.amazon.com/Apple-iPhone-11-64GB-Unlocked/dp/B07ZPKF8RG/ref=sr_1_13?dchild=1&keywords=iphone+12&qid=1631721363&sr=8-13) on amazon.

(2) Collect the top 10000 User Reviews of the film [Shang-Chi and the Legend of the Ten Rings](https://www.imdb.com/title/tt9376612/reviews?ref_=tt_sa_3) from IMDB.

(3) Collect all the reviews of the top 100 most popular software from [G2](https://www.g2.com/) or [Capterra](https://www.capterra.com/)

(4) Collect the abstracts of the top 10000 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(5) Collect all the information of the 904 narrators in the [Densho Digital Repository](https://ddr.densho.org/narrators/).

(6) Collect the top 10000 tweets by using hashtag ["#blacklivesmatter"](https://twitter.com/hashtag/blacklivesmatter) from Twitter. 


In [2]:
# Write your code here


# Write your code here


#importing Libraries
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup as BS
import re
import numpy as np
import pandas as pd

#function calling
def get_tablelist(soup):
    reviews = soup.find_all("div", {"class": "lister-item-content"})

    review_data = []

    for review in reviews:

        try:
            review_content = review.find("div", {"class": "text show-more__control"}).text.strip()
        except:
            review_content = review.find("div", {"class": "text show-more__control clickable"}).text.strip()

        review_data.append(review_content)
    return review_data

b_url = 'https://www.imdb.com/'
url = 'https://www.imdb.com/title/tt9376612/reviews?ref_=tt_sa_3'
res = requests.get(url)
soup = BS(res.text, 'html.parser')
res.encoding = 'utf-8'
allTablelist=[]
tablelist = get_tablelist(soup)
for i in tablelist:
  allTablelist.append(i)

load_more = soup.select(".load-more-data")
flag = True
if len(load_more):
    ajaxurl = load_more[0]['data-ajaxurl']
    b_url = b_url + ajaxurl + "?ref_=undefined&paginationKey="
    try:
        key = load_more[0]['data-key']
    except KeyError:
        flag = False
else:
    flag = False

while flag:
    url = b_url + key
    res = requests.get(url)
    res.encoding = 'utf-8'
    soup = BS(res.text, 'html.parser')
    tablelist2 = get_tablelist(soup)
    if len(allTablelist) == 1235:
      break
    for i in tablelist2:
        allTablelist.append(i)
    load_more = soup.select(".load-more-data")
    if len(load_more):
        key = load_more[0]['data-key']
    else:
        flag = False

data = np.array(allTablelist)
data = pd.DataFrame(data, columns=['movie_Reviews'])
data.shape
data.to_csv('imdb_reviews.csv')
data


,movie_Reviews
0,I got to admit I wasn't particularly blown awa...
1,The characters are a bit incoherent and straig...
2,Destin Daniel Cretton's 'Shang-Chi and the Leg...
3,The story was riddled with holes and plot issu...
4,Everything comes in it beautiful science ficti...
...,...
2467,Shang-Chi and the Legend of Ten Rings and a Dr...
2468,"For me personally, that was a 10 out of 10 mov..."
2469,All the events were totally predictable. I cou...
2470,Watch it! And you will know.never felt this vi...


In [ ]:
data.to_csv('shangchi_marvel_reviews.csv', index=False)

# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [3]:
# Write your code here


# punctuation removal
data['movie_Reviews'] = data['movie_Reviews'].str.replace('[^\w\s]','')
data['movie_Reviews'].head()


C:\Users\gurug\AppData\Local\Temp/ipykernel_4440/2865359497.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  data['movie_Reviews'] = data['movie_Reviews'].str.replace('[^\w\s]','')


0    I got to admit I wasnt particularly blown away...
1    The characters are a bit incoherent and straig...
2    Destin Daniel Crettons ShangChi and the Legend...
3    The story was riddled with holes and plot issu...
4    Everything comes in it beautiful science ficti...
Name: movie_Reviews, dtype: object

In [5]:
#remove special characters
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [6]:
#remove special characters
data['movie_Reviews'] = data['movie_Reviews'].apply(lambda x: " ".join(x.strip(string.punctuation) for x in x.split()))  
data['movie_Reviews']

0       I got to admit I wasnt particularly blown away...
1       The characters are a bit incoherent and straig...
2       Destin Daniel Crettons ShangChi and the Legend...
3       The story was riddled with holes and plot issu...
4       Everything comes in it beautiful science ficti...
                              ...                        
2467    ShangChi and the Legend of Ten Rings and a Dra...
2468    For me personally that was a 10 out of 10 movi...
2469    All the events were totally predictable I coul...
2470    Watch it And you will knownever felt this vibe...
2471    As its quite clear that marvel has a standard ...
Name: movie_Reviews, Length: 2472, dtype: object

In [8]:
#remove stopwords
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['movie_Reviews'] = data['movie_Reviews'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
data['movie_Reviews'].head()

0    I got admit I wasnt particularly blown away an...
1    The characters bit incoherent straight contrad...
2    Destin Daniel Crettons ShangChi Legend Ten Rin...
3    The story riddled holes plot issues Since Awkw...
4    Everything comes beautiful science fiction bea...
Name: movie_Reviews, dtype: object

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gurug\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
# Write your code here
#Lower Casing
data['movie_Reviews'] = data['movie_Reviews'].apply(lambda x: " ".join(x.lower() for x in x.split()))
data['movie_Reviews'].head()

0    i got admit i wasnt particularly blown away an...
1    the characters bit incoherent straight contrad...
2    destin daniel crettons shangchi legend ten rin...
3    the story riddled holes plot issues since awkw...
4    everything comes beautiful science fiction bea...
Name: movie_Reviews, dtype: object

In [11]:
#stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
data['movie_Reviews'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

0    i got admit i wasnt particularli blown away an...
1    the charact bit incoher straight contradict ch...
2    destin daniel cretton shangchi legend ten ring...
3    the stori riddl hole plot issu sinc awkwafina ...
4    everyth come beauti scienc fiction beauti myth...
Name: movie_Reviews, dtype: object

In [16]:
#lemmatization
from textblob import Word
nltk.download('wordnet')
data['movie_Reviews'] = data['movie_Reviews'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data['movie_Reviews'].head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gurug\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0    i got admit i wasnt particularly blown away an...
1    the character bit incoherent straight contradi...
2    destin daniel crettons shangchi legend ten rin...
3    the story riddled hole plot issue since awkwaf...
4    everything come beautiful science fiction beau...
Name: movie_Reviews, dtype: object

# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [18]:
# Write your code here

# Write your code here
nltk.download('averaged_perceptron_tagger')







[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\gurug\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

In [19]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gurug\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [20]:
# 3(1) Parts of speech(POS) Tagging
from nltk import word_tokenize, pos_tag, pos_tag_sents
data['POSTags'] =  pos_tag_sents( data['movie_Reviews'].apply(word_tokenize).tolist() )
print(data)

pos = data['POSTags'].to_list()
pos
nouns = 0
for x in pos:
  for a,b in x:
    if(b.startswith('N')):
       nouns+=1
output = "nouns:{}".format(nouns)
print(output)
verb = 0
for x in pos:
  for a,b in x:
    if(b.startswith('V')):
       verb+=1
output1 = "verb:{}".format(verb)
print(output1)
adverb = 0
for x in pos:
  for a,b in x:
    if(b.startswith('R')):
       adverb+=1
output2 = "adverb:{}".format(adverb)
print(output2)
adj = 0
for x in pos:
  for a,b in x:
    if(b.startswith('J')):
       adj+=1
output3 = "adj:{}".format(adj)
print(output3)

                                          movie_Reviews  \
0     i got admit i wasnt particularly blown away an...   
1     the character bit incoherent straight contradi...   
2     destin daniel crettons shangchi legend ten rin...   
3     the story riddled hole plot issue since awkwaf...   
4     everything come beautiful science fiction beau...   
...                                                 ...   
2467  shangchi legend ten ring dragon not using heli...   
2468  for personally 10 10 movie i enjoyed whole way...   
2469  all event totally predictable i could barely k...   
2470  watch and knownever felt vibe last time watchi...   
2471  a quite clear marvel standard formula making o...   

                                                POSTags  
0     [(i, NN), (got, VBD), (admit, RB), (i, JJ), (w...  
1     [(the, DT), (character, NN), (bit, NN), (incoh...  
2     [(destin, NN), (daniel, NN), (crettons, NNS), ...  
3     [(the, DT), (story, NN), (riddled, VBD), (hole...  
4

2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

Consituency parsing - displays syntactic structure where the words are organized into nested constituents. Both are helpful in word processing systems and grammar correction.

Dependency parsing - displays grammatical structure where each word represents a node and followed by links to its dependents. It displays the relation between words.

In [22]:
#entity recognition
!pip install spacy

In [23]:
!pip3 install en_core_web_sm
!python -m spacy download en

ERROR: Could not find a version that satisfies the requirement en_core_web_sm (from versions: none)
ERROR: No matching distribution found for en_core_web_sm


[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [24]:
# 3(2) Name entity Recognition
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = spacy.load("en_core_web_sm")
data['movie_Reviews']
list = data['movie_Reviews'].tolist()
list[0]
for x in list:
   doc = (nlp(x))
   print([(X.text, X.label_) for X in doc.ents])

[('third', 'ORDINAL'), ('china', 'GPE'), ('mandarin', 'LANGUAGE')]
[('2021', 'CARDINAL'), ('ww84', 'PRODUCT'), ('one', 'CARDINAL')]
[('destin daniel', 'ORG')]
[('awkwafinas', 'GPE'), ('zero', 'CARDINAL'), ('third', 'ORDINAL'), ('ben kingsleys', 'PERSON'), ('asian', 'NORP'), ('710', 'CARDINAL')]
[]
[('one', 'CARDINAL'), ('2000', 'CARDINAL'), ('one', 'CARDINAL')]
[('20', 'CARDINAL'), ('50', 'CARDINAL'), ('5', 'CARDINAL'), ('chinese', 'NORP'), ('chinese', 'LANGUAGE'), ('1', 'CARDINAL'), ('chinese', 'NORP'), ('summer', 'DATE')]
[('shangchi one', 'PERSON'), ('asian', 'NORP'), ('shangchi differentshangchi', 'PERSON'), ('s', 'ORG'), ('characterswen wu', 'PERSON'), ('third', 'ORDINAL'), ('chinese', 'NORP'), ('chinese', 'NORP')]
[('shangchi boring', 'PERSON')]
[('first', 'ORDINAL'), ('one', 'CARDINAL'), ('chinese', 'NORP')]
[('hong kong', 'GPE'), ('indonesia', 'GPE'), ('thailand', 'GPE'), ('last twenty year', 'DATE')]
[('2', 'CARDINAL'), ('2nd', 'ORDINAL'), ('2nd', 'ORDINAL'), ('510', 'CARDINAL